In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
from datetime import datetime
import logging
import socket  # Added for getting host IP

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# 1. Data Preprocessing
def load_and_preprocess_data():
    try:
        df = pd.read_csv('covid_19_india.csv')
        logger.debug(f"Unique raw state names: {df['State/UnionTerritory'].unique()}")
        logger.info("CSV loaded successfully")
        
        df['State/UnionTerritory'] = df['State/UnionTerritory'].str.strip() \
            .str.lower() \
            .str.replace(r'\s+', ' ', regex=True) \
            .str.title()
        
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y', errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.fillna(0)
        
        numeric_columns = ['ConfirmedIndianNational', 'ConfirmedForeignNational', 'Cured', 'Deaths', 'Confirmed']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        df['Active'] = df['Confirmed'] - df['Cured'] - df['Deaths']
        logger.info("Data preprocessing completed")
        return df
    except Exception as e:
        logger.error(f"Error in load_and_preprocess_data: {str(e)}")
        raise

# 2. Data Analysis Functions
def get_state_wise_data(df, date):
    latest_data = df[df['Date'] == date]
    state_data = latest_data.groupby('State/UnionTerritory').agg({
        'Confirmed': 'sum',
        'Deaths': 'sum',
        'Cured': 'sum',
        'Active': 'sum'
    }).reset_index()
    return state_data

def get_time_series_data(df, state):
    return df[df['State/UnionTerritory'] == state]

# 3. Initialize Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "COVID-19 India Dashboard"

# 4. Load data
try:
    df = load_and_preprocess_data()
    states = sorted(df['State/UnionTerritory'].unique())
    dates = sorted(df['Date'].unique())
except Exception as e:
    logger.error(f"Failed to load data: {str(e)}")
    raise

# 5. App Layout with adjusted dropdown styling and new graphs
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("COVID-19 India Dashboard", className="text-center mt-4 mb-4"),
            html.P("Dashboard showing COVID-19 statistics across India", className="text-center text-muted")
        ])
    ]),

    dbc.Row([
        dbc.Col([
            html.Label("Select Date:"),
            dcc.DatePickerSingle(
                id='date-picker',
                min_date_allowed=df['Date'].min().date(),
                max_date_allowed=df['Date'].max().date(),
                initial_visible_month=df['Date'].max().date(),
                date=df['Date'].max().date(),
                display_format='DD/MM/YYYY'
            ),
        ], width=4),

        dbc.Col([
            html.Label("Select State:"),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in states],
                value='Kerala',
                clearable=False,
                style={'width': '100%', 'max-width': '600px'}
            ),
        ], width=4),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Confirmed", className="card-title"),
                html.H2(id="total-confirmed", className="card-text"),
            ])
        ], color="primary", inverse=True), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Deaths", className="card-title"),
                html.H2(id="total-deaths", className="card-text"),
            ])
        ], color="danger", inverse=True), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Recovered", className="card-title"),
                html.H2(id="total-recovered", className="card-text"),
            ])
        ], color="success", inverse=True), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Active Cases", className="card-title"),
                html.H2(id="active-cases", className="card-text"),
            ])
        ], color="warning", inverse=True), width=3),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col([dcc.Graph(id='state-map')], width=6),
        dbc.Col([dcc.Graph(id='time-series')], width=6),
    ]),

    dbc.Row([
        dbc.Col([dcc.Graph(id='pie-chart')], width=6),
        dbc.Col([dcc.Graph(id='bar-chart')], width=6),
    ]),

    # New Row for Additional Graphs
    dbc.Row([
        dbc.Col([dcc.Graph(id='active-trend')], width=6),  # New graph for active cases trend
        dbc.Col([dcc.Graph(id='stacked-bar')], width=6),  # New graph for stacked bar
    ]),
], fluid=True)

# 6. Callbacks
@app.callback(
    [Output('total-confirmed', 'children'),
     Output('total-deaths', 'children'),
     Output('total-recovered', 'children'),
     Output('active-cases', 'children'),
     Output('state-map', 'figure'),
     Output('time-series', 'figure'),
     Output('pie-chart', 'figure'),
     Output('bar-chart', 'figure'),
     Output('active-trend', 'figure'),  # New output for active trend
     Output('stacked-bar', 'figure')],  # New output for stacked bar
    [Input('date-picker', 'date'),
     Input('state-dropdown', 'value')]
)
def update_dashboard(selected_date, selected_state):
    try:
        logger.debug(f"Callback triggered with date: {selected_date}, state: {selected_state}")
        selected_date = pd.to_datetime(selected_date)
        logger.debug(f"Converted selected_date: {selected_date}")
        
        state_data = get_state_wise_data(df, selected_date)
        time_data = get_time_series_data(df, selected_state)
        logger.debug(f"State data shape: {state_data.shape}, Time data shape: {time_data.shape}")
        
        if state_data.empty or time_data.empty:
            logger.warning("Empty data detected")
            empty_fig = go.Figure()
            empty_fig.update_layout(title="No Data Available")
            return (
                "N/A", "N/A", "N/A", "N/A",
                empty_fig, empty_fig, empty_fig, empty_fig,
                empty_fig, empty_fig  # Return empty figures for new graphs
            )

        total_confirmed = f"{int(state_data['Confirmed'].sum()):,}"
        total_deaths = f"{int(state_data['Deaths'].sum()):,}"
        total_recovered = f"{int(state_data['Cured'].sum()):,}"
        total_active = f"{int(state_data['Active'].sum()):,}"
        logger.debug("Summary stats calculated")

        map_fig = px.choropleth(
            state_data,
            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
            featureidkey='properties.ST_NM',
            locations='State/UnionTerritory',
            color='Confirmed',
            color_continuous_scale='Reds',
            title='Confirmed Cases by State',
            range_color=[0, state_data['Confirmed'].max()]
        )
        map_fig.update_geos(
            fitbounds="locations",
            visible=False,
            projection_type="mercator",
            center={"lat": 20.5937, "lon": 78.9629}
        )
        map_fig.update_layout(
            margin={"r":0,"t":40,"l":0,"b":0},
            height=400
        )
        logger.debug("Map figure created")

        time_fig = go.Figure()
        time_fig.add_trace(go.Scatter(x=time_data['Date'], y=time_data['Confirmed'], name='Confirmed', line=dict(color='blue')))
        time_fig.add_trace(go.Scatter(x=time_data['Date'], y=time_data['Deaths'], name='Deaths', line=dict(color='red')))
        time_fig.add_trace(go.Scatter(x=time_data['Date'], y=time_data['Cured'], name='Recovered', line=dict(color='green')))
        time_fig.update_layout(
            title=f'COVID-19 Cases in {selected_state}',
            xaxis_title='Date',
            yaxis_title='Cases',
            height=400
        )
        logger.debug("Time series figure created")

        pie_data_row = state_data[state_data['State/UnionTerritory'] == selected_state]
        if pie_data_row.empty:
            pie_fig = go.Figure()
            pie_fig.update_layout(title=f'No Data for {selected_state}')
        else:
            pie_data = pie_data_row.iloc[0]
            pie_fig = px.pie(
                values=[pie_data['Confirmed'], pie_data['Deaths'], pie_data['Cured']],
                names=['Confirmed', 'Deaths', 'Recovered'],
                title=f'Case Distribution in {selected_state}',
                height=400
            )
        logger.debug("Pie chart figure created")

        bar_fig = px.bar(
            state_data.sort_values('Confirmed', ascending=False).head(10),
            x='State/UnionTerritory',
            y='Confirmed',
            title='Top 10 States by Confirmed Cases',
            height=400
        )
        bar_fig.update_layout(xaxis={'tickangle': 45})
        logger.debug("Bar chart figure created")

        # New Active Cases Trend Graph
        active_fig = go.Figure()
        active_fig.add_trace(go.Scatter(
            x=time_data['Date'],
            y=time_data['Active'],
            name='Active Cases',
            line=dict(color='orange')
        ))
        active_fig.update_layout(
            title=f'Active Cases Trend in {selected_state}',
            xaxis_title='Date',
            yaxis_title='Active Cases',
            height=400
        )
        logger.debug("Active trend figure created")

        # New Stacked Bar Chart
        top_states = state_data.sort_values('Confirmed', ascending=False).head(5)
        stacked_fig = go.Figure(data=[
            go.Bar(name='Confirmed', x=top_states['State/UnionTerritory'], y=top_states['Confirmed']),
            go.Bar(name='Recovered', x=top_states['State/UnionTerritory'], y=top_states['Cured']),
            go.Bar(name='Deaths', x=top_states['State/UnionTerritory'], y=top_states['Deaths']),
            go.Bar(name='Active', x=top_states['State/UnionTerritory'], y=top_states['Active'])
        ])
        stacked_fig.update_layout(
            barmode='stack',
            title='COVID-19 Metrics for Top 5 States',
            xaxis_title='State',
            yaxis_title='Cases',
            height=400,
            xaxis={'tickangle': 45}
        )
        logger.debug("Stacked bar figure created")

        return (total_confirmed, total_deaths, total_recovered, total_active,
                map_fig, time_fig, pie_fig, bar_fig,
                active_fig, stacked_fig)
    
    except Exception as e:
        logger.error(f"Error in callback: {str(e)}")
        empty_fig = go.Figure()
        empty_fig.update_layout(title="Error Occurred")
        return (
            "Error", "Error", "Error", "Error",
            empty_fig, empty_fig, empty_fig, empty_fig,
            empty_fig, empty_fig
        )

# 7. Run the app and display URL
if __name__ == '__main__':
    port = 8050
    try:
        # Get the local IP address
        host = socket.gethostbyname(socket.gethostname())
        url = f"http://{host}:{port}"
        print(f"Dashboard is running at: {url}")
        app.run(debug=True, port=port, host='0.0.0.0')  # Allow external access
    except Exception as e:
        logger.error(f"Failed to start server: {str(e)}")
        print("Failed to start the dashboard. Please check the logs for details.")